In [123]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
import json
import re
from pprint import pprint
import warnings

In [5]:
len(warns)

73934

In [16]:
warns[:1000]

'validate.py:208: UserWarning: too many inter object linkages for 0.png\n  warnings.warn("too many inter object linkages for %s" % image_name)\nvalidate.py:69: UserWarning: too many inter_object_linkage_count for 0.png: 5\n  warnings.warn(message)\nvalidate.py:208: UserWarning: too many inter object linkages for 1.png\n  warnings.warn("too many inter object linkages for %s" % image_name)\nvalidate.py:69: UserWarning: too many inter_object_linkage_count for 1.png: 4\n  warnings.warn(message)\nvalidate.py:109: UserWarning: invalid intraobject label origins count for image 1005.png: L1 \n  warnings.warn("invalid intraobject label origins count for image %s: %s " % (image_name, rel_id))\nvalidate.py:211: UserWarning: wrong number of blobs for 1010.png: 7\n  warnings.warn("wrong number of blobs for %s: %s" % (image_name, blob_count(annotations)))\nvalidate.py:214: UserWarning: wrong number of blobs for 1010.png: 7\n  warnings.warn("wrong number of blobs for %s: %s" % (image_name, object_cou

In [17]:
split_seq = ['reported found in\s*', 'reported[ly]* present in\s*',
             'the.*oils*', 'originally identified in',
             'reportedly found in\s*', 'also reported found in', '  ', ' and other ', ' and ', ',', ';',
             'also ', 'in\s+']
ingr_list = [raw_ing_string.lower()]
for term in split_seq:
    nested_list = map(lambda x: re.split(term, x), ingr_list)

NameError: name 'raw_ing_string' is not defined

In [21]:
patt = 'validate.py:\d+: UserWarning: .*warnings.warn'
re.split(patt, warns[:1000])

['validate.py:208: UserWarning: too many inter object linkages for 0.png\n  warnings.warn("too many inter object linkages for %s" % image_name)\nvalidate.py:69: UserWarning: too many inter_object_linkage_count for 0.png: 5\n  warnings.warn(message)\nvalidate.py:208: UserWarning: too many inter object linkages for 1.png\n  warnings.warn("too many inter object linkages for %s" % image_name)\nvalidate.py:69: UserWarning: too many inter_object_linkage_count for 1.png: 4\n  warnings.warn(message)\nvalidate.py:109: UserWarning: invalid intraobject label origins count for image 1005.png: L1 \n  warnings.warn("invalid intraobject label origins count for image %s: %s " % (image_name, rel_id))\nvalidate.py:211: UserWarning: wrong number of blobs for 1010.png: 7\n  warnings.warn("wrong number of blobs for %s: %s" % (image_name, blob_count(annotations)))\nvalidate.py:214: UserWarning: wrong number of blobs for 1010.png: 7\n  warnings.warn("wrong number of blobs for %s: %s" % (image_name, object_co

In [47]:
warning_pattern = re.compile('validate.py:\d+:\s*(.+)')
re.findall(warning_pattern, warns[:1000])

['UserWarning: too many inter object linkages for 0.png',
 'UserWarning: too many inter_object_linkage_count for 0.png: 5',
 'UserWarning: too many inter object linkages for 1.png',
 'UserWarning: too many inter_object_linkage_count for 1.png: 4',
 'UserWarning: invalid intraobject label origins count for image 1005.png: L1 ',
 'UserWarning: wrong number of blobs for 1010.png: 7',
 'UserWarning: wrong number of blobs for 1010.png: 7']

In [97]:
warning_pattern = re.compile('validate.py:\d+:\s+UserWarning:\s+(.+)(for)+?\s+[0-9]+\.png')
re.findall(warning_pattern, warns)

[('too many inter object linkages ', 'for'),
 ('too many inter_object_linkage_count ', 'for'),
 ('too many inter object linkages ', 'for'),
 ('too many inter_object_linkage_count ', 'for'),
 ('wrong number of blobs ', 'for'),
 ('wrong number of blobs ', 'for'),
 ('too many inter object linkages ', 'for'),
 ('too many inter_object_linkage_count ', 'for'),
 ('too many inter object linkages ', 'for'),
 ('too many inter_object_linkage_count ', 'for'),
 ('too many inter object linkages ', 'for'),
 ('too many inter_object_linkage_count ', 'for'),
 ('too many inter object linkages ', 'for'),
 ('too many inter_object_linkage_count ', 'for'),
 ('too many arrow_count ', 'for'),
 ('too many intra_object_linkage_count ', 'for'),
 ('too many inter_object_linkage_count ', 'for'),
 ('invalid number of relationships ', 'for'),
 ('invalid number of relationships ', 'for'),
 ('invalid number of relationships ', 'for'),
 ('invalid number of relationships ', 'for'),
 ('too many arrow_count ', 'for'),
 ('t

In [98]:
def read_annotations(image_name):
    filename = "/Users/schwenk/wrk/vision/datasets/shining3/annotations/%s.json" % image_name
    with open(filename) as f:
        return json.loads(f.read())

In [107]:
annotations = read_annotations('1698.png')

In [114]:
def safe_get(annotations, path, default):
    accum = annotations
    for p in path:
        accum = accum.get(p)
        if accum is None:
            return default
    return accum


def find_object_for_element(annotations, element_id):
    if 'objects' in annotations:
        key_map = dict(T='text', B='blobs', R='regions')
        objects = list(filter(lambda x: element_id in x.get(key_map[element_id[0]], []), list(annotations['objects'].values())))
        if len(objects) == 1:
            return objects[0]

    return None

def find_relationships_for_element(annotations, element_id):
    labels = safe_get(annotations, ['relationships', 'intraObject', 'label'], {}).values()
    intra_linkages = safe_get(annotations, ['relationships', 'intraObject', 'linkage'], {}).values()
    inter_linkages = safe_get(annotations, ['relationships', 'interObject', 'linkage'], {}).values()
    relationship_ids = []
    for relationships in [labels, intra_linkages, inter_linkages]:
        for relationship in relationships:
            if element_id in relationship['origin'] or element_id in relationship['destination']:
                relationship_ids.append(relationship['id'])

    return relationship_ids

In [128]:
annotations['text'].get('T0', [])

{'category': 'regionDescriptionNoArrow',
 'id': 'T0',
 'parentObject': 'O0',
 'rectangle': [[254, 100], [302, 114]],
 'refersTo': ['B0'],
 'replacementText': 'F',
 'value': 'Volcano'}

In [191]:
safe_get(annotations, ['arrows'], {})

{'A0': {'arrowHeads': [],
  'category': 'regionLabel',
  'id': 'A0',
  'polygon': [[446, 223], [431, 227], [433, 237], [447, 233], [446, 222]]},
 'A1': {'arrowHeads': ['H1'],
  'category': 'regionLabel',
  'id': 'A1',
  'polygon': [[452, 178], [442, 189], [448, 193], [458, 183], [452, 178]]},
 'A2': {'arrowHeads': [],
  'category': 'regionLabel',
  'id': 'A2',
  'polygon': [[250, 179], [241, 187], [244, 192], [255, 186], [250, 179]]},
 'A3': {'arrowHeads': [],
  'category': 'regionLabel',
  'id': 'A3',
  'polygon': [[126, 27], [139, 28], [139, 150], [126, 150], [124, 27]]},
 'A4': {'arrowHeads': [],
  'category': 'regionLabel',
  'id': 'A4',
  'polygon': [[375, 28], [384, 28], [386, 118], [370, 115], [372, 29]]},
 'A5': {'arrowHeads': [],
  'category': 'regionLabel',
  'id': 'A5',
  'polygon': [[592, 47],
   [606, 48],
   [603, 109],
   [605, 128],
   [591, 126],
   [591, 48]]},
 'A6': {'arrowHeads': ['H0'],
  'category': 'regionLabel',
  'id': 'A6',
  'polygon': [[631, 194], [651, 194

In [190]:
annotations.keys()

dict_keys(['relationships', 'objects', 'arrows', 'blobs', 'arrowHeads', 'regions', 'text'])

In [189]:
find_object_for_element(annotations, 'R2')

KeyError: 'R2'

AttributeError: 'list' object has no attribute 'istype'

In [136]:
annotations['text'].get('T0', [])

{'category': 'regionDescriptionNoArrow',
 'id': 'T0',
 'parentObject': 'O0',
 'rectangle': [[254, 100], [302, 114]],
 'refersTo': ['B0'],
 'replacementText': 'F',
 'value': 'Volcano'}

In [195]:
pprint(annotations)

{'arrowHeads': {'H0': {'id': 'H0',
                       'orientation': 157.5,
                       'rectangle': [[634, 197], [640, 203]]},
                'H1': {'id': 'H1',
                       'orientation': 123.75,
                       'rectangle': [[447, 185], [451, 189]]}},
 'arrows': {'A0': {'arrowHeads': [],
                   'category': 'regionLabel',
                   'id': 'A0',
                   'polygon': [[446, 223],
                               [431, 227],
                               [433, 237],
                               [447, 233],
                               [446, 222]]},
            'A1': {'arrowHeads': ['H1'],
                   'category': 'regionLabel',
                   'id': 'A1',
                   'polygon': [[452, 178],
                               [442, 189],
                               [448, 193],
                               [458, 183],
                               [452, 178]]},
            'A2': {'arrowHeads': [],
         

Error in schema --'refersTo' is a required property-- for, 1248 occurrences


Error in schema --Additional properties are not allowed ('replacementText' was unexpected)-- for, 1171 occurrences


Error in schema --'category' is a required property-- for, 793 occurrences


Error in schema --'ownObject' is not one of ['objectLabel', 'regionLabel', 'imageTitle', 'misc', 'sectionTitle', 'arrowDescriptor', 'imageCaption', 'regionDescriptionNoArrow']-- for, 206 occurrences


Error in schema --Additional properties are not allowed ('textLinkage' was unexpected)-- for, 100 occurrences

In [ ]:
# Use a Draft3Validator
try:
    jsonschema.Draft3Validator(json.loads(schema)).validate(json.loads(data))
except jsonschema.ValidationError as e:
    print e.message
This will just report the first error it catches. Interestingly, you can also use lazy validation to report all validation errors:
# Lazily report all errors in the instance
try:
    v = jsonschema.Draft3Validator(json.loads(schema))
    for error in sorted(v.iter_errors(json.loads(data)), key=str):
        print(error.message)
except jsonschema.ValidationError as e:
    print e.message


In [196]:
safe_get(annotations, ['relationships', 'intraObject', 'label'], {})

{'L10': {'category': 'regionDescriptionNoArrow',
  'destination': ['B0'],
  'id': 'L10',
  'origin': ['T7']},
 'L7': {'category': 'regionDescriptionNoArrow',
  'destination': ['B0'],
  'id': 'L7',
  'origin': ['T6']},
 'L8': {'category': 'regionDescriptionNoArrow',
  'destination': ['B0'],
  'id': 'L8',
  'origin': ['T9']},
 'L9': {'category': 'regionDescriptionNoArrow',
  'destination': ['B0'],
  'id': 'L9',
  'origin': ['T0']}}

In [192]:
safe_get(annotations, ['relationships', 'intraObject', 'label'], {})

{'L10': {'category': 'regionDescriptionNoArrow',
  'destination': ['B0'],
  'id': 'L10',
  'origin': ['T7']},
 'L7': {'category': 'regionDescriptionNoArrow',
  'destination': ['B0'],
  'id': 'L7',
  'origin': ['T6']},
 'L8': {'category': 'regionDescriptionNoArrow',
  'destination': ['B0'],
  'id': 'L8',
  'origin': ['T9']},
 'L9': {'category': 'regionDescriptionNoArrow',
  'destination': ['B0'],
  'id': 'L9',
  'origin': ['T0']}}

In [141]:
annotations['text'].get('T0', [])['category']

'regionDescriptionNoArrow'

In [171]:
all_label_origins

['T0', 'T7', 'T9', 'T6']

In [131]:
image_name = 'blah'

In [184]:
all_label_origins = []
for rlabel, descr in safe_get(annotations, ['relationships', 'intraObject', 'label'], {}).items():
    this_text_box = descr['category'] 
    if this_text_box == 'regionDescriptionNoArrow':
        origin = descr['origin']
        all_label_origins.append(origin[0])

        if len(origin) != 1 or origin[0][0] !='T':
            warnings.warn("invalid intraobject region origin %s" % image_name)
        if annotations['text'].get(origin[0], [])['category'] != 'regionDescriptionNoArrow':
            warnings.warn("invalid intraobject region label for image %s" % image_name)

for text_box, descr in safe_get(annotations, ['text'], {}).items():
    if descr.get('category', []) == 'regionDescriptionNoArrow':
        if text_box not in all_label_origins:
            print(text_box)
            warnings.warn("invalid intraobject region label for image %s" % image_name)

In [111]:
pprint(annotations)

{'arrowHeads': {'H0': {'id': 'H0',
                       'orientation': 157.5,
                       'rectangle': [[634, 197], [640, 203]]},
                'H1': {'id': 'H1',
                       'orientation': 123.75,
                       'rectangle': [[447, 185], [451, 189]]}},
 'arrows': {'A0': {'arrowHeads': [],
                   'category': 'regionLabel',
                   'id': 'A0',
                   'polygon': [[446, 223],
                               [431, 227],
                               [433, 237],
                               [447, 233],
                               [446, 222]]},
            'A1': {'arrowHeads': ['H1'],
                   'category': 'regionLabel',
                   'id': 'A1',
                   'polygon': [[452, 178],
                               [442, 189],
                               [448, 193],
                               [458, 183],
                               [452, 178]]},
            'A2': {'arrowHeads': [],
         

In [125]:
pprint(annotations['text'])

{'T0': {'category': 'regionDescriptionNoArrow',
        'id': 'T0',
        'parentObject': 'O0',
        'rectangle': [[254, 100], [302, 114]],
        'refersTo': ['B0'],
        'replacementText': 'F',
        'value': 'Volcano'},
 'T1': {'category': 'regionLabel',
        'id': 'T1',
        'parentObject': 'O0',
        'rectangle': [[106, 10], [163, 26]],
        'replacementText': 'I',
        'value': 'Lava flow'},
 'T2': {'category': 'regionLabel',
        'id': 'T2',
        'parentObject': 'O0',
        'rectangle': [[252, 174], [330, 190]],
        'replacementText': 'C',
        'value': 'Volcanic pipe'},
 'T3': {'category': 'regionLabel',
        'id': 'T3',
        'parentObject': 'O0',
        'rectangle': [[351, 12], [411, 28]],
        'replacementText': 'E',
        'value': 'Lava flow'},
 'T4': {'category': 'regionLabel',
        'id': 'T4',
        'parentObject': 'O0',
        'rectangle': [[546, 16], [648, 49]],
        'replacementText': 'H',
        'value': 'L

In [179]:
all_label_origins = []
for rlabel, descr in safe_get(annotations, ['relationships', 'intraObject', 'label'], {}).items():
    this_text_box = descr['category']
    if not this_text_box == 'regionDescriptionNoArrow':
        warnings.warn("invalid intraobject region label for image %s" % image_name)
    origin = descr['origin']
    all_label_origins.append(origin[0])

    if len(origin) != 1 or origin[0][0] !='T':
        warnings.warn("invalid intraobject region origin %s" % image_name)
    if annotations['text'].get(origin[0], [])['category'] != 'regionDescriptionNoArrow':
        warnings.warn("invalid intraobject region label for image %s" % image_name)

for text_box, descr in safe_get(annotations, ['text'], {}).items():
    if descr['category'] == 'regionDescriptionNoArrow':
        if text_box not in all_label_origins:
            warnings.warn("invalid intraobject region label for image %s" % image_name)